# SD212: Graph mining
## Lab 2: Graph structure

In this lab, you will learn to:
* plot the degree distribution and observe the [power-law](https://en.wikipedia.org/wiki/Power_law#Power-law_probability_distributions),
* estimate the [diameter](https://en.wikipedia.org/wiki/Distance_(graph_theory)) of a graph and observe the [small-world](https://en.wikipedia.org/wiki/Small-world_network) property,
* estimate the [clustering coefficient](https://en.wikipedia.org/wiki/Clustering_coefficient) of a graph.

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt

In [ ]:
from sknetwork.data import load_netset, erdos_renyi, cyclic_graph, karate_club
from sknetwork.path import distance, diameter, shortest_path
from sknetwork.utils import bipartite2undirected
from sknetwork.topology import Triangles
from sknetwork.visualization import svg_graph

## Data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr) collection for details):
* Openflights (graph)
* WikiVitals (digraph)
* Cinema (bigraph)

In [ ]:
openflights = load_netset('openflights')

In [ ]:
# focus on degrees, not weights
adjacency = (openflights.adjacency > 0)

In [ ]:
wikivitals = load_netset('wikivitals')

In [ ]:
cinema = load_netset('cinema')

## 1. Power law

We first focus on the power law degree distribution.

We consider the empirical distributions of:
* degrees in the Openflights graph,
* out-degrees and in-degrees in the WikiVitals graph.

## To do

For each of these distributions:
* Compute the [coefficient of variation](https://en.wikipedia.org/wiki/Coefficient_of_variation) and compare it to that of a Poisson distribution with the same mean.
* Plot the [tail distribution](https://en.wikipedia.org/wiki/Cumulative_distribution_function#Complementary_cumulative_distribution_function_(tail_distribution)) in log-log scale and compare it to that of an equivalent [Erdos-Rényi](https://en.wikipedia.org/wiki/Erdős–Rényi_model) graph (same number of nodes, same average degree).
* Conclude: do you consider that the degree distributions of these graphs have a power law?

In [ ]:
n = adjacency.shape[0]
degrees = adjacency.dot(np.ones(n))

In [ ]:
plt.loglog(np.sort(degrees), 1 - np.arange(n) / n, lw = 3)
plt.xlabel('Degree')
plt.ylabel('Probability')
plt.show()

## 2. Small-world

You will now make a few experiments highlighting the small-world property.

## To do

* Complete the following function computing the distances from any given node in number of hops.
* Check your code with the function ``distance``.
* Give ones of the longest shortest paths starting from **Paris Charles de Gaulle** airport in Openflights.<br> 
You might use the function ``shortest_path``.<br>
List the corresponding airports and display the path.

In [ ]:
def get_distances(adjacency: sparse.csr_matrix, source: int) -> np.ndarray:
    '''Get distances from a source node in number of hops.
    
    Parameters
    ----------
    adjacency: 
        Adjacency matrix of the graph.
    source:
        Source node
    
    Returns
    -------
    distances:
        Vector of distances.
    '''
    n = adjacency.shape[0]
    distances = np.inf * np.ones(n)
    distances[source] = 0
    reached = np.zeros(n).astype(bool)
    reached[source] = True
    stop = False
    t = 0
    while not stop:
        t += 1
        reached_new = adjacency.T.dot(reached)
        # to be modified   
        stop = True
    return distances

In [ ]:
adjacency = openflights.adjacency > 0
distances = get_distances(adjacency, 0)

In [ ]:
cdg = 622
airports = openflights.names
print(airports[cdg])

In [ ]:
def get_adjacency_path(adjacency, path):
    '''Return a path as an adjacency matrix (subset of edges).'''
    row = path[:-1]
    col = path[1:]
    data = np.ones(len(row), dtype=bool)
    adjacency_path = sparse.csr_matrix((data, (row, col)), shape=adjacency.shape)
    return adjacency_path

In [ ]:
# a random shortest path
source, dest = 900, 2500
print(airports[source], '->', airports[dest])
path = shortest_path(adjacency, source, dest)

In [ ]:
# representation as a sparse matrix
adjacency_path = get_adjacency_path(adjacency, path)

In [ ]:
# display
position = openflights.position
image = svg_graph(adjacency_path, position, directed=True, height=400, width=800, 
                  node_size=1, edge_width=3, edge_color='blue')

In [ ]:
SVG(image)

## To do

* Which fraction of the airports can be reached from **Paris Charles de Gaulle** in 1, 2, 3 flights?
* Which fraction of the articles can be reached from **Paris** in 1, 2, 3 clicks on Wikipedia Vitals?

## To do

The following function estimates the [diameter](https://en.wikipedia.org/wiki/Distance_(graph_theory)) of a graph.

* Write a function giving the corresponding path (longest shortest path).
* Give a path of **length 12** in Openflights (with the airport names) and display it.
* Give a path of **length 6** in WikiVitals (with the names of the articles).

In [ ]:
def estimate_diameter(adjacency: sparse.csr_matrix, n_samples=100) -> int:
    '''Estimate the diameter of a graph (longest shortest path).
    
    Parameters
    ----------
    adjacency: 
        Adjacency matrix of the graph.
    n_samples:
        Number of samples.
    
    Returns
    -------
    diameter:
        Estimation of the diameter. 
    '''    
    n = adjacency.shape[0]
    # start from random nodes
    nodes = np.random.choice(n, size=n_samples)
    distances = distance(adjacency.T, sources=nodes)
    distances[distances == np.inf] = -1
    # get the corresponding furthest nodes
    nodes_far = distances.argmax(axis=1)    
    distances = distance(adjacency, sources=nodes_far)
    distances[distances == np.inf] = -1
    # take the longest shortest path from these nodes 
    diameter = distances.max()
    return int(diameter)

In [ ]:
estimate_diameter(adjacency)

In [ ]:
def estimate_diameter_path(adjacency: sparse.csr_matrix, n_samples=100) -> int:
    '''Return a path of length equal to the (estimated) diameter of a graph.
    
    Parameters
    ----------
    adjacency: 
        Adjacency matrix of the graph.
    n_samples:
        Number of samples.
    
    Returns
    -------
    path:
        List of nodes. 
    '''    
    # to be modified
    source = 0
    dest = 0
    path = shortest_path(adjacency, source, dest)
    return path

In [ ]:
path = estimate_diameter_path(adjacency)

## To do

* Compute the [Bacon number](https://fr.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) of all actors (**without** building the co-starring actor graph).
* Plot the corresponding histogram.
* Which fraction of actors have a Bacon number of 3 at most?

In [ ]:
biadjacency = cinema.biadjacency

In [ ]:
adjacency = bipartite2undirected(biadjacency)

In [ ]:
n_row = biadjacency.shape[0]

In [ ]:
names = cinema.names_col

In [ ]:
{i:name for i,name in enumerate(names) if 'Bacon' in name}

In [ ]:
bacon = n_row + 3465

## 3. Clustering coefficient

The computation of the clustering coefficient of a graph is time-consuming.
You will now learn to **estimate** it.

## To do


* Complete the following function to estimate the clustering coefficient of a graph.
* Check your code on the triange, then on the karate club graph using ``Triangles``.
* Estimate the clustering coefficient of Openflights and WikiVitals (seen as an undirected graph) and compare with that of equivalent Erdos-Rényi graphs (same number of nodes, same average degree).

**Hint:** A triangle is... a cycle of length 3

In [ ]:
def get_clustering_coefficient(adjacency: sparse.csr_matrix, n_samples: int = 100) -> float:
    '''Get the clustering coefficient of a graph.
    
    Parameters
    ----------
    adjacency: 
        Adjacency matrix of the graph (binary).
    
    Returns
    -------
    clustering_coef:
        Clustering coefficient.
    '''
    clustering_coef = 0
    # sample nodes
    n = adjacency.shape[0]
    if n_samples < n:
        nodes = np.random.choice(n, size=n_samples, replace=False)
    else:
        nodes = np.arange(n)
    # compute the clustering coefficient for these samples
    # to be modified
    clustering_coef = 0
    return clustering_coef

In [ ]:
triangle = cyclic_graph(3)

In [ ]:
algo = Triangles()

In [ ]:
algo.fit(triangle)

In [ ]:
algo.clustering_coef_

In [ ]:
get_clustering_coefficient(triangle)